<a href="https://colab.research.google.com/github/saadahmed388/projects/blob/main/logos_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import models,layers
import matplotlib.pyplot as plt

IMAGE_SIZE = 256
BATCH_SIZE = 32
CHANNELS = 3
EPOCHS =50


In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
   "/content/drive/MyDrive/Car_Brand_Logos/Train",
   shuffle=True,
   image_size = (IMAGE_SIZE,IMAGE_SIZE),
   batch_size = BATCH_SIZE
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/drive/MyDrive/Car_Brand_Logos/Test",
    shuffle=True,
    image_size = (IMAGE_SIZE,IMAGE_SIZE),
    batch_size = BATCH_SIZE
) 

#test_ds = test.take(round(len(test)*0.5))
#val_ds = test.skip(len(test_ds))

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

Found 2456 files belonging to 8 classes.
Found 391 files belonging to 8 classes.


In [ ]:
len(train_ds)

79

In [ ]:
import os
import cv2
import imghdr

def check_images( s_dir, ext_list):
    bad_images=[]
    bad_ext=[]
    s_list= os.listdir(s_dir)
    for klass in s_list:
        klass_path=os.path.join (s_dir, klass)
        print ('processing class directory ', klass)
        if os.path.isdir(klass_path):
            file_list=os.listdir(klass_path)
            for f in file_list:               
                f_path=os.path.join (klass_path,f)
                tip = imghdr.what(f_path)
                if ext_list.count(tip) == 0:
                  bad_images.append(f_path)
                if os.path.isfile(f_path):
                    try:
                        img=cv2.imread(f_path)
                        shape=img.shape
                    except:
                        print('file ', f_path, ' is not a valid image file')
                        bad_images.append(f_path)
                else:
                    print('*** fatal error, you a sub directory ', f, ' in class directory ', klass)
        else:
            print ('*** WARNING*** you have files in ', s_dir, ' it should only contain sub directories')
    return bad_images, bad_ext

source_dir =r'/content/drive/MyDrive/Car_Brand_Logos/Test'
good_exts=['jpg', 'png', 'jpeg', 'gif', 'bmp' ] # list of acceptable extensions
bad_file_list, bad_ext_list=check_images(source_dir, good_exts)
if len(bad_file_list) !=0:
    print('improper image files are listed below')
    for i in range (len(bad_file_list)):
        print (bad_file_list[i])
else:
    print(' no improper image files were found')

    

processing class directory  toyota
processing class directory  volkswagen
processing class directory  hyundai
processing class directory  mercedes
processing class directory  skoda
processing class directory  lexus
processing class directory  mazda
processing class directory  opel
 no improper image files were found


In [ ]:
for i in bad_file_list:
  os.remove(i)

In [ ]:
train_size = 0.8

train_ds = dataset.take(round(len(dataset)*train_size))
test_ds = dataset.skip(len(train_ds))

val_ds = test_ds.take(round(len(test_ds)*0.5))
test_ds = test_ds.skip(len(val_ds))

def get_dataset_partitions_tf(ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
    
    ds_size = len(ds)
    
    if shuffle:
        ds = ds.shuffle(shuffle_size , seed = 12)
        
    train_size = int(train_split *ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds

train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset)

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
resize_and_rescale = tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(IMAGE_SIZE,IMAGE_SIZE),
    layers.experimental.preprocessing.Rescaling(1.0/255)
    ])

data_augmentation = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(0.2),
    
])

input_shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
n_classes = 8

model = models.Sequential([
    resize_and_rescale,
    data_augmentation,
    layers.Conv2D(32, (3,3), activation = 'relu', input_shape = input_shape),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation = 'relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation = 'relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation = 'relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation = 'relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation = 'relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(64, activation ='relu'),
    layers.Dense(n_classes, activation ='softmax'),
])
model.build(input_shape=input_shape)

In [ ]:
model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_12 (Sequential)  (32, 256, 256, 3)         0         
                                                                 
 sequential_13 (Sequential)  (32, 256, 256, 3)         0         
                                                                 
 conv2d_24 (Conv2D)          (32, 254, 254, 32)        896       
                                                                 
 max_pooling2d_24 (MaxPoolin  (32, 127, 127, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_25 (Conv2D)          (32, 125, 125, 64)        18496     
                                                                 
 max_pooling2d_25 (MaxPoolin  (32, 62, 62, 64)         0         
 g2D)                                                

In [ ]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    train_ds,
    epochs=50,
    batch_size=BATCH_SIZE,
    verbose = 1,
    validation_data = test_ds
)

77/77 [==============================] - 6s 72ms/step - loss: 0.2894 - accuracy: 0.9059 - val_loss: 1.6393 - val_accuracy: 0.6854
Epoch 42/50
77/77 [==============================] - 6s 72ms/step - loss: 0.2981 - accuracy: 0.8998 - val_loss: 1.7022 - val_accuracy: 0.6726
Epoch 43/50
77/77 [==============================] - 5s 71ms/step - loss: 0.2974 - accuracy: 0.8982 - val_loss: 1.8007 - val_accuracy: 0.6496
Epoch 44/50
77/77 [==============================] - 6s 72ms/step - loss: 0.2703 - accuracy: 0.9055 - val_loss: 1.7859 - val_accuracy: 0.6803
Epoch 45/50
77/77 [==============================] - 6s 72ms/step - loss: 0.2893 - accuracy: 0.9039 - val_loss: 1.9134 - val_accuracy: 0.6496
Epoch 46/50
77/77 [==============================] - 6s 74ms/step - loss: 0.3387 - accuracy: 0.8888 - val_loss: 1.6973 - val_accuracy: 0.6880
Epoch 47/50
77/77 [==============================] - 6s 72ms/step - loss: 0.2966 - accuracy: 0.9039 - val_loss: 1.5657 - val_accuracy: 0.6777
Epoch 48/50
77/77 

In [ ]:
model.save('/content/drive/MyDrive/logo_scanning_model/model_1')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/logo_scanning_model/model_1/assets
